# ViT Model Fine-tuning & Deployment on Inferentia2/Trainium

ViT モデルは、テキストベースのタスク用に設計された transformer アーキテクチャに基づくビジュアルモデルです。

ImageNet-21K データセットで事前学習された　ViT モデルを、Beans データセットでファインチューニングします。
このモデルでは、数エポック学習することで、Beans(葉)の健康状態を3つのカテゴリに分類して予測可能です。 

## 事前準備
本 notebookは Neuron 2.13.2 環境下で動作確認しています

In [1]:
!pip install -U pip
!pip install -U transformers==4.31.0 accelerate evaluate gradio

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com


In [2]:
!pip list | grep "neuron\|torch\|transformers"

aws-neuronx-runtime-discovery 2.9
libneuronxla                  0.5.413
neuronx-cc                    2.8.0.25+a3ad0f342
neuronx-hwm                   2.8.0.3+2b7c6da39
torch                         1.13.1
torch-neuronx                 1.13.1.1.9.0
torch-xla                     1.13.1+torchneuron8
torchvision                   0.14.1
transformers                  4.31.0


In [3]:
!dpkg --list | grep neuron

hi  aws-neuronx-collectives                2.15.16.0-db4e2d9a9               amd64        neuron_ccom built using CMake
hi  aws-neuronx-dkms                       2.11.9.0                          amd64        aws-neuronx driver in DKMS format.
hi  aws-neuronx-oci-hook                   2.2.16.0                          amd64        neuron_oci_hook built using CMake
hi  aws-neuronx-runtime-lib                2.15.14.0-279f319f2               amd64        neuron_runtime built using CMake
hi  aws-neuronx-tools                      2.12.2.0                          amd64        Neuron profile and debug tools


In [4]:
!sudo rmmod neuron; sudo modprobe neuron

## Trainer API を使用した トレーニング（ファインチューニング）実行
Transformers には Trainer という便利なクラスがあり、Torch Neuron からも利用可能です。 ここでは Trainer API を利用してトレーニングを実行していきます。


In [5]:
!pygmentize run_image_classification.py

#!/usr/bin/env python
# coding=utf-8
# Copyright 2021 The HuggingFace Inc. team. All rights reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and

import logging
import os
import sys
from dataclasses import dataclass, field
from typing import Optional

import evaluate
import numpy as np
import torch
from datasets import load_dataset
from PIL import Image
from torchvision.transforms import (
    CenterCrop,
    Compose,
    Normalize,
    RandomHorizontalFlip,
    RandomResizedCrop,
    Resize,
    ToTensor,
)

import transfor

- HuggingFace transformers API を使用して ViT モデルをファインチューニングします。
- Neuron コア上で実行されるデータ型は、より効率を高めるために `fp32` ではなく `bf16` を使用します。
- コンパイルされたモデルアーティファクトが保存されるモデルキャッシュディレクトリ（`./compiler_cache`）を指定します。
- PyTorchの `torchrun` コマンドを使用してトレーニングジョブを起動します。
- Inferentia2 (もしくは Trainium) チップを １つ搭載した　Inf2.xlarge (もしくは Trn1.2xlarge) 上での実行を想定しています。各チップは ２ つの Neuron コアを搭載しているため `num_workers=2` と設定、結果、トレーニングジョブは 2つの Neuron コア上で実行されます。
- モデルを 10 エポック学習し、エポックごとにモデルのチェックポイントを保存します。保存できるチェックポイントは 1 つまでです。ロギング情報は 10 回ごとに出力します。
-　`./output` ディレクトリには、ファインチューニングで生成されたモデルの重み、Config、その他のアーティファクトが格納されます

In [6]:
%%time
!XLA_USE_BF16=1 NEURON_CC_FLAGS="--cache_dir=./compiler_cache" \
torchrun --nproc_per_node=2 run_image_classification.py \
--model_name_or_path "google/vit-base-patch16-224-in21k" \
--dataset_name "beans" \
--do_train \
--do_eval \
--num_train_epochs 10 \
--per_device_train_batch_size 16 \
--per_device_eval_batch_size 16 \
--learning_rate 2e-5 \
--logging_strategy steps \
--logging_steps 10 \
--save_strategy epoch \
--save_total_limit 1 \
--seed 1337 \
--remove_unused_columns False \
--overwrite_output_dir \
--output_dir "output"

*****************************************
Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
*****************************************
Please use the TrainiumTrainer from optimum[neuron] instead of the Transformers library to perform training on AWS Trainium instances. More information here: https://github.com/huggingface/optimum-neuron
Please use the TrainiumTrainer from optimum[neuron] instead of the Transformers library to perform training on AWS Trainium instances. More information here: https://github.com/huggingface/optimum-neuron
torch.distributed process group is initialized, but parallel_mode != ParallelMode.DISTRIBUTED. In order to use Torch DDP, launch your script with `python -m torch.distributed.launch
torch.distributed process group is initialized, but parallel_mode != ParallelMode.DISTRIBUTED. In order to use Torch

コンパイル時間を含んだ学習には `inf2.8xlarge` 上で実行した場合で 25\~30分程度かかります.
2度目以降の実行ではコンパイル済みのキャッシュが利用可能なため、1\~2分程度で学習が完了します。

`neuron_parallel_compile` コマンドを利用したコンパイル時間の削減方法については、[日本語BERTモデルのサンプル](https://github.com/AWShtokoyo/aws-ml-jp/tree/main/frameworks/aws-neuron-jp/bertj_finetuning_classification)を参照下さい。


これで　AWS Inferentia2 上での ViT モデルのファインチューニングに成功しました。 
`pytorch_model.bin` という名前のファインチューニングされた重みを持つモデル、`Trainer` の状態、モデル設定ファイル（`config.json`） を含むファイルのリストが表示されます。

In [7]:
!ls -l ./output/

total 335268
-rw-rw-r-- 1 ubuntu ubuntu      1614 Sep  7 05:14 README.md
-rw-rw-r-- 1 ubuntu ubuntu       329 Sep  7 05:14 all_results.json
drwxrwxr-x 2 ubuntu ubuntu      4096 Sep  7 05:14 checkpoint-330
-rw-rw-r-- 1 ubuntu ubuntu       845 Sep  7 05:14 config.json
-rw-rw-r-- 1 ubuntu ubuntu       185 Sep  7 05:14 eval_results.json
-rw-rw-r-- 1 ubuntu ubuntu       325 Sep  7 05:14 preprocessor_config.json
-rw-rw-r-- 1 ubuntu ubuntu 343269037 Sep  7 05:14 pytorch_model.bin
-rw-rw-r-- 1 ubuntu ubuntu       165 Sep  7 05:14 train_results.json
-rw-rw-r-- 1 ubuntu ubuntu      4548 Sep  7 05:14 trainer_state.json
-rw-rw-r-- 1 ubuntu ubuntu      3963 Sep  7 05:14 training_args.bin


# ViT 推論

In [8]:
from PIL import Image
import requests
import torch
import torch_neuronx
from transformers import ViTImageProcessor, ViTForImageClassification

# Create the feature extractor and model
checkpoint_dir = './output/'
print(f"Create model from provided checkpoint: {checkpoint_dir}")
feature_extractor = ViTImageProcessor.from_pretrained(checkpoint_dir)
model = ViTForImageClassification.from_pretrained(checkpoint_dir, torchscript=True)
model.eval()

# Get an example input
url = "https://datasets-server.huggingface.co/assets/beans/--/default/test/0/image/image.jpg"
image = Image.open(requests.get(url, stream=True).raw)
inputs = feature_extractor(images=image, return_tensors="pt")
example = (inputs['pixel_values'],)

# Run inference on CPU
output_cpu = model(*example)

Create model from provided checkpoint: ./output/


## 推論実行のためのモデルの事前コンパイル

推論を AWS Inferentia2　または AWS Trainium 上で実行するためには、モデルを`torch_neuronx.trace` APIを用いて事前にトレース（コンパイル）する必要があります。トレース（コンパイル）した結果は保存することで再利用可能です。

In [9]:
%%time
# Compile the model for neuron
print(f"Compile model for neuron with torch tracing ...")
model_neuron = torch_neuronx.trace(model, example)

# Save the TorchScript for inference deployment
filename = 'vit-model-neuron.pt'
torch.jit.save(model_neuron, filename)
print(f"Save compiled model as: {filename}")

Compile model for neuron with torch tracing ...
Save compiled model as: vit-model-neuron.pt
CPU times: user 7.65 s, sys: 2.31 s, total: 9.96 s
Wall time: 40.4 s


期待通りの出力が得られるかどうか　CPU上での推論結果と比較します。

In [ ]:
# Load the TorchScript compiled model
print(f"Load compiled model: {filename}")
model_neuron = torch.jit.load(filename)

# Run inference using the Neuron model
print(f"Run inference on the test image: {url}")
output_neuron = model_neuron(*example)

# Compare the results
print(f"--- Compare Neuron output against CPU output ----")
print(f"CPU tensor:            {output_cpu[0][0][0:10]}")
print(f"Neuron tensor:         {output_neuron[0][0][0:10]}")
print(f"CPU prediction:    {model.config.id2label[output_cpu[0].argmax(-1).item()]}")
print(f"Neuron prediction: {model.config.id2label[output_neuron[0].argmax(-1).item()]}")

## Gradio API を用いた推論デモ

モデルサービスのデモをセットアップする簡易な方法は、Gradio API を使用することです。画像をアップロードしてモデルに与え、推論結果を確認します。

In [ ]:
from PIL import Image

from torchvision.transforms import (
    CenterCrop,
    Compose,
    Normalize,
    RandomHorizontalFlip,
    RandomResizedCrop,
    Resize,
    ToTensor,
)

id2label = {0: 'angular_leaf_spot　(角葉スポット)', 1: 'bean_rust　(豆さび病)', 2: 'healthy　(健康)'}

def predict(raw_image):
    size = (224, 224)
    image_mean = [0.5, 0.5, 0.5]
    image_std = [0.5, 0.5, 0.5]
    normalize = Normalize(mean=image_mean, std=image_std)
    
    _val_transforms = Compose(
        [
            Resize(size),
            CenterCrop(size),
            ToTensor(),
            normalize,
        ]
    )
    
    transformed_image = _val_transforms(raw_image.convert("RGB"))
    batched_transformed_image = transformed_image.unsqueeze(0)
    
    with torch.no_grad():
        prediction = model_neuron(batched_transformed_image)
        pred = id2label[prediction[0].argmax(-1).item()]
    return pred

In [ ]:
import gradio as gr

demo = gr.Interface(fn=predict,
             inputs=gr.Image(type="pil"),
             outputs="text",
             examples=[
                 'image_samples/healthy_test.21.jpg',
                 'image_samples/angular_leaf_spot_test.21.jpg',
                 'image_samples/bean_rust_test.34.jpg'])

demo.launch(share=True)